# Cifrado y descifrado. Criptosistema McEliece.


In [1]:
set_random_seed(1234)

Trabajaremos con códigos Goppa binarios, $p=2$; elegimos $m=4$ 

In [2]:
rango = 3
m = 4
N = 2^m
K_.<a> = GF(2) # cuerpo con el que trabajaremos
F.<a> = GF(2^m) # cuerpo de los coef. del pol. Goppa

In [3]:
PR = PolynomialRing(F,'X') # anillo de polinomios
X = PR.gen()
g = X^3+X+1 # Polinomio de Goppa
L = [a^i for i in range(N)]

In [4]:
show(K_)
show(F)
show(PR)
show(g)
show(L)

Finite Field of size 2

Finite Field in a of size 2^4

Univariate Polynomial Ring in X over Finite Field in a of size 2^4

X^3 + X + 1

[1,
 a,
 a^2,
 a^3,
 a + 1,
 a^2 + a,
 a^3 + a^2,
 a^3 + a + 1,
 a^2 + 1,
 a^3 + a,
 a^2 + a + 1,
 a^3 + a^2 + a,
 a^3 + a^2 + a + 1,
 a^3 + a^2 + 1,
 a^3 + 1,
 1]

## Funciones auxiliares

In [5]:
# -------------------
# Funcion auxiliar que permite descomponer un polinomio en irreducibles
# -------------------
def descomponer_polinomio(p):
    # El siguiente metodo permite descomponer un polinomio p en factores irreducibles p(z) = p0 (z) + z p1 (z)
    # Entrada: Polinomio p
    Phi1 = p.parent()

    p0 = Phi1([sqrt(c) for c in p.list()[0::2]])
    p1 = Phi1([sqrt(c) for c in p.list()[1::2]])
    return (p0,p1)

In [6]:
# -------------------
# Algoritmo Extendido de Euclides: Obtener MCD y los s,t que lo generan.
# -------------------

def algoritmo_euclides_extendido(self, other):
    delta = self.degree() # grado de polinomio 1

    if other.is_zero(): # si el polinomio introducido es 0
        ring = self.parent() # comprobamos el cuerpo en el que trabajamos
        return self, R.one(), R.zero() # mcd = mismo polinomio y devuelve un uno (s) y un cero (t) en el cuerpo que trabajamos.

    # mcd (a,b) = as+bt
    
    ring = self.parent() # comprobamos el cuerpo en el que trabajamos
    a = self # guardamos una copia del primer polinomio 1 (self)
    b = other # guardamos una copia del segundo polinomio (other)

    s = ring.one() # guardamos en s el uno del anillo
    t = ring.zero() # guardamos en t el cero del anillo

    resto0 = a
    resto1 = b

    while true:
        cociente,resto_auxiliar = resto0.quo_rem(resto1) # La funcion quo_rem de Sage devuelve el cociente y el resto. Que guardamos en Q y ring.
        resto0 = resto1
        resto1 = resto_auxiliar

        s = t
        t = s - t*cociente

        if resto1.degree() <= floor((delta-1)/2) and resto0.degree() <= floor((delta)/2):
             break

    V = (resto0-a*s)//b
    coeficiente_lider = resto0.leading_coefficient() # guardamos el coeficiente lider del resto 0

    return resto0/coeficiente_lider, s/coeficiente_lider, V/coeficiente_lider


In [7]:
# -------------------
# Funcion que calcula la inversa de un polinomio utilizando el algoritmo de euclides de Sage
# -------------------
def inversa_g(p,g):
    (d,u,v) = xgcd(p,g)
    return u.mod(g)

In [8]:
# -------------------
# Funcion de decodificacion de Patterson
# -------------------
def decodePatterson(y):

    alpha = vector(H*y) # Calculamos primero el vector alpha con los elementos primitivos.

    # Consideramos nuestras matrices T,Y,Z definidas así como nuestro polinomio irreducible g
    
    polinomioS = PR(0) # Inicializa como el polinomio 0 del anillo
    for i in range(len(alpha)):
        polinomioS = polinomioS + alpha[i]*(X^(len(alpha)-i-1)) # Lo vamos rellenando con los alpha

    vector_g = descomponer_polinomio(g) # Guardamos en vector_g el par de polinomios irreducibles
    w = ((vector_g[0])*inversa_g(vector_g[1],g)).mod(g)
    vector_t = descomponer_polinomio(inversa_g(polinomioS,g) + X)

    R = (vector_t[0]+(w)*(vector_t[1])).mod(g)

    (a11,b11,c11) = algoritmo_euclides_extendido(g,R)


    sigma = a11^2+X*(c11^2) # Definimos el polinomio sigma

    # Vamos comprobando uno a uno los coeficientes de sigma
    # para asi determinar el conjunto de posiciones de error E - {i tal que e_i es distinto de 0}
    for i in range(N):
        if (sigma(a^i)==0):
            y[i] = y[i] + 1
    return y


## Construcción de la matriz G

In [9]:
# -------------------
# Matriz T
# -------------------

T = matrix(F,rango,rango)
for i in range(rango):
    count = rango - i
    for j in range(rango):
        if i > j:
            T[i,j]=g.list()[count]
            count = count + 1
        if i < j:
            T[i,j] = 0
        if i == j:
            T[i,j] = 1

In [10]:
print ("Matriz T: ")
show(T)

Matriz T: 


[1 0 0]
[0 1 0]
[1 0 1]

In [11]:
# -------------------
# Matriz V
# -------------------

V = matrix([[L[j]^i for j in range(N)] for i in range(rango)])
print ("Matriz V: ")
show(V)

print("Conjunto del Cuerpo de Goppa: ")
show(L)

Matriz V: 


[                1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1]
[                1                 a               a^2               a^3             a + 1           a^2 + a         a^3 + a^2       a^3 + a + 1           a^2 + 1           a^3 + a       a^2 + a + 1     a^3 + a^2 + a a^3 + a^2 + a + 1     a^3 + a^2 + 1           a^3 + 1                 1]
[                1               a^2             a + 1         a^3 + a^2           a^2 + 1       a^2 + a + 1 a^3 + a^2 + a + 1           a^3 + 1                 a               a^3           a^2 + a       a^3 + a + 1           a^3 + a     a^3 + a^2 + a     a^3 + a^2 + 1                 1]

Conjunto del Cuerpo de Goppa: 


[1,
 a,
 a^2,
 a^3,
 a + 1,
 a^2 + a,
 a^3 + a^2,
 a^3 + a + 1,
 a^2 + 1,
 a^3 + a,
 a^2 + a + 1,
 a^3 + a^2 + a,
 a^3 + a^2 + a + 1,
 a^3 + a^2 + 1,
 a^3 + 1,
 1]

In [12]:
# -------------------
# Matriz D
# -------------------

D = diagonal_matrix([1/g(L[i]) for i in range(N)])
print ("Matriz D: ")
show(D)

Matriz D: 


[            1             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0]
[            0       a^2 + 1             0             0             0             0             0             0             0             0             0             0             0             0             0             0]
[            0             0             a             0             0             0             0             0             0             0             0             0             0             0             0             0]
[            0             0             0 a^3 + a^2 + a             0             0             0             0             0             0             0             0             0             0             0             0]
[            0             0             0             0           a^2             0             0             0             0             0             0             0             0             0             0             0]
[            0             0             0             0             0   a^2 + a + 1             0             0             0             0             0             0             0             0             0             0]
[            0             0             0             0             0             0   a^3 + a + 1             0             0             0             0             0             0             0             0             0]
[            0             0             0             0             0             0             0   a^2 + a + 1             0             0             0             0             0             0             0             0]
[            0             0             0             0             0             0             0             0         a + 1             0             0             0             0             0             0             0]
[            0             0             0             0             0             0             0             0             0 a^3 + a^2 + 1             0             0             0             0             0             0]
[            0             0             0             0             0             0             0             0             0             0       a^2 + a             0             0             0             0             0]
[            0             0             0             0             0             0             0             0             0             0             0       a^2 + a             0             0             0             0]
[            0             0             0             0             0             0             0             0             0             0             0             0       a^3 + 1             0             0             0]
[            0             0             0             0             0             0             0             0             0             0             0             0             0   a^2 + a + 1             0             0]
[            0             0             0             0             0             0             0             0             0             0             0             0             0             0       a^2 + a             0]
[            0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             1]

In [13]:
# -------------------
# Matriz H
# -------------------

H = T*V*D
print ("Matriz H: ")
show(H)

Matriz H: 


[                1           a^2 + 1                 a     a^3 + a^2 + a               a^2       a^2 + a + 1       a^3 + a + 1       a^2 + a + 1             a + 1     a^3 + a^2 + 1           a^2 + a           a^2 + a           a^3 + 1       a^2 + a + 1           a^2 + a                 1]
[                1           a^3 + a               a^3           a^3 + 1         a^3 + a^2                 1     a^3 + a^2 + 1               a^2 a^3 + a^2 + a + 1       a^3 + a + 1                 1                 a     a^3 + a^2 + a           a^2 + 1             a + 1                 1]
[                0                 a               a^2           a^3 + a             a + 1                 1               a^3     a^3 + a^2 + 1           a^2 + 1 a^3 + a^2 + a + 1                 1           a^3 + 1         a^3 + a^2       a^3 + a + 1     a^3 + a^2 + a                 0]

In [14]:
# -------------------
# Matriz H Goppa K
# -------------------

H_Goppa_K = matrix(K_, m*H.nrows(),H.ncols())

for i in range(H.nrows()):
    for j in range(H.ncols()):
        be = bin(eval(H[i,j]._int_repr()))[2:];
        be = '0'*(m-len(be))+be; be = list(be);
        H_Goppa_K[m*i:m*(i+1),j]=vector(map(int,be));

print ("Matriz H_Goppa_K: ")
show(H_Goppa_K)

Matriz H_Goppa_K: 


[0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0]
[0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 0]
[0 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0]
[1 1 0 0 0 1 1 1 1 1 0 0 1 1 0 1]
[0 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0]
[0 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0]
[0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 0]
[1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1]
[0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 0]
[0 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0]
[0 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0]
[0 0 0 0 1 1 0 1 1 1 1 1 0 1 0 0]

In [15]:
# -------------------
# Matriz Kernel
# -------------------

Krnl = H_Goppa_K.right_kernel();
print ("Matriz Krnl: ")
show(Krnl)

Matriz Krnl: 


Vector space of degree 16 and dimension 4 over Finite Field of size 2
Basis matrix:
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0]
[0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1]
[0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 1]

In [16]:
# -------------------
# Matriz G
# -------------------

G = Krnl.basis_matrix();
print ("Matriz G: ")
show(G)

Matriz G: 


[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0]
[0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1]
[0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 1]

## Construcción matrices S y P, completando la clave privada (G, S, P)

In [17]:
# -------------------
# Matriz S
# -------------------

S = random_matrix(GF(2), N-m*rango)

while (S.determinant()==0):
    S = random_matrix(GF(2), N-m*rango)

print ("Matriz S: ")
show(S)

Matriz S: 


[1 1 0 1]
[0 0 0 1]
[0 1 1 0]
[0 1 0 0]

In [18]:
# -------------------
# Matriz P
# -------------------

rng = range(N)

P = matrix(GF(2),N);

for i in range(N):
    p = floor(len(rng)*random());
    P[i,rng[p]]=1;
    rng=[*rng[:p], *rng[p+1:]];

print ("Matriz P: ")
show(P)

Matriz P: 


[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]

## Matriz de clave pública G'

In [19]:
# -------------------
# Matriz G'
# -------------------
G_prima = S*G*P

print ("Matriz G_prima: ")
show(G_prima)

Matriz G_prima: 


[0 0 1 1 0 0 0 1 1 1 1 0 0 1 0 1]
[1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 0]
[0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 1]
[1 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1]

## Veamos un ejemplo de envío de un mensaje

Supongamos que Alice tiene clave pública G', y Bob quiere enviarle un mensaje.

### Cifrado
Proceso de cifrado:

1. Bob quiere cifrar un mensaje binario $\textbf{m}$ de longitud igual al número de filas que $G'$

In [20]:
m = vector(K_,[randint(0,1) for _ in range(G_prima.nrows())]) # mensaje cualquiera
print("Vector m");
show(m)

Vector m


(1, 0, 1, 0)

2. Bob calcula el vector $\textbf{c} = \textbf{m}G'$

In [21]:
c = m*G_prima
print("Vector c");
show(c)

Vector c


(0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0)

3. Bob genera el vector de errores $\textbf{e}$ con peso $\textbf{wt(e)} = \textbf{t}$

In [22]:
e = vector(K_,N)
# se introducen errores
e[8] = 1
e[9] = 1
print("Vector de errores e");
show(e)

Vector de errores e


(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0)

4. Bob calcula el texto cifrado $\textbf{c}' = \textbf{c} + \textbf{e}$

In [23]:
c_p = c + e
print("Vector codificado c'"); show(c_p)

Vector codificado c'


(0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0)

### Descifrado

Proceso de descifrado:


1. Alice calcula $\hat{\textbf{c}}=\textbf{c}'P^{-1}$

In [24]:
c_h = c_p*(P.inverse()); show(c_h)

(1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1)

2. Alice usa el algoritmo de Patterson para obtener $\textbf{mSG}$ ya que la matriz $\textbf{SG}$ es una matriz generadora de código Goppa y tanto $\textbf{e}$ como $\textbf{e}P^{-1}$ tienen el mismo peso, siendo este $\textbf{t}$

In [25]:
mSG = decodePatterson(c_h)
show(mSG)

(1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1)

3. Alice obtiene $\textbf{m'} = \textbf{mS}$ resolviendo el sistema de ecuaciones correspondiente

In [26]:
mS = G.transpose()\mSG

4. Alice descifra el mensaje a partir de la matriz inversa de $\textbf{S}$ y ya puede leer el mensaje original

In [27]:
m_recibido = mS * S.inverse()
show(m_recibido)

(1, 0, 1, 0)

Comprobamos que es el mismo mensaje

In [28]:
m==m_recibido

True